In [1]:
import numpy as np
from datetime import datetime

from CaliPytion.core import Standard, SignalType, Sample

## Create Samples

In [2]:
def signal_relation(x: np.ndarray):
    param_a = 0.0123
    return x*param_a

analyte_concs = np.linspace(0,200,11)
signals = signal_relation(analyte_concs)

samples = [Sample(concentration=analyte_conc, signal=signal, conc_unit="mM") 
           for analyte_conc, signal 
           in zip(analyte_concs, signals)]

In [3]:
standard = Standard(
    species_id="s0",
    name="test substrance",
    ph=3,
    temperature=25,
    temperature_unit='C',
    signal_type=SignalType.ABSORBANCE,
    samples=samples,
    wavelength=420
)

In [4]:
print(standard)

Standard
├── id = 0b6a5bc4-9d34-4d0d-94cc-b6520280401b
├── species_id = s0
├── name = test substrance
├── ph = 3.0
├── temperature = 25.0
├── temperature_unit = C
├── wavelength = 420.0
├── signal_type = Absorbance
└── samples
    ├── 0
    │   └── Sample
    │       ├── id = ce2cabfb-283d-4dc2-8fc5-7d02842719b0
    │       ├── concentration = 0.0
    │       ├── conc_unit = mM
    │       └── signal = 0.0
    ├── 1
    │   └── Sample
    │       ├── id = 1cfe516e-0f20-496f-ad1b-3734a2fbb899
    │       ├── concentration = 20.0
    │       ├── conc_unit = mM
    │       └── signal = 0.246
    ├── 2
    │   └── Sample
    │       ├── id = 04e0bf1b-b2e1-40ef-b3e0-0655952eafa2
    │       ├── concentration = 40.0
    │       ├── conc_unit = mM
    │       └── signal = 0.492
    ├── 3
    │   └── Sample
    │       ├── id = ca0c9639-bd94-408a-a6ff-6ff2b461e33e
    │       ├── concentration = 60.0
    │       ├── conc_unit = mM
    │       └── signal = 0.738
    ├── 4
    │   └── Sample
   

## Create calibration model

In [5]:
%reload_ext autoreload
%autoreload 2

from CaliPytion.tools import Calibrator

In [6]:
cal = Calibrator.from_standard(standard)

In [7]:
cal.fit_models()

,AIC,R squared,RMSD
Model Name,,,
linear,-6330,1.0000,0.0000
quadratic,-581,1.0000,0.0000
cubic,-577,1.0000,0.0000


In [8]:
cal.models[2].calculate_concentrations([0.45, float("nan"), 1.12, 2])

[36.58536585385819, nan, 91.05691056917989, 162.60162601638226]

In [9]:
def _get_residuals(
    self, concentrations: List[float], signals: List[float]
) -> np.ndarray:
    return np.array(signals) - self.signal_callable(concentrations, **self._params)


NameError: name 'List' is not defined

In [ ]:
cal.visualize()

AttributeError: 'CalibrationModel' object has no attribute '_get_residuals'

In [ ]:
print(c.models[0])

CalibrationModel
├── id = 1c315bbe-493b-4310-8d8d-b6261e332709
├── name = linear
├── signal_equation = a * s0
├── parameters
│   └── 0
│       └── Parameter
│           ├── id = 1198f8d2-0725-412d-8031-7eedf521bf1d
│           ├── name = a
│           └── init_value = 0.1
├── calibration_range
│   └── CalibrationRange
│       └── id = 81b14dcb-6b4a-4198-b6bc-af5bdd14c84a
└── statistics
    └── FitStatistics
        └── id = 7507e1ce-4ab5-4c81-8f75-edfbe1baab34



In [ ]:
c.fit_models()

,AIC,R squared,RMSD
Model Name,,,
linear,-2301,1.0000,0.0000
quadratic,-201,1.0000,0.0000
cubic,-199,1.0000,0.0000


## Generate artificial data for a `Standard`

## Initialize a `Calibrator` from a `Standard`

By defining a cutoff value, samples with a higher signal than the defined cutoff are ignored for the calibration.
For concentration calculation, the model does not extrapolate beyond the cutoff value by default.

In [ ]:
# initialize calibrator
calibrator = Calibrator.from_standard(standard, cutoff=2.5)

print(calibrator)

species_id='s0' name=None concentrations=[0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0, 140.0, 160.0, 180.0, 200.0] conc_unit='mM' signals=[0.0, 0.246, 0.492, 0.738, 0.984, 1.23, 1.476, 1.722, 1.968, 2.214, 2.46] models=[CalibrationModel(id='1c315bbe-493b-4310-8d8d-b6261e332709', name='linear', signal_equation='a * s0', parameters=[Parameter(id='e94dee92-7cfa-45e6-9d9e-f155478bb487', name='a', value=None, init_value=0.1, standard_error=None, lower_bound=None, upper_bound=None)], was_fitted=True, calibration_range=CalibrationRange(id='ad914325-0f08-41a3-b0e1-2cb2be97d46d', conc_lower=0.0, conc_upper=3.0, signal_lower=0.0, signal_upper=3.0), statistics=FitStatistics(id='943e7cfb-463d-43b3-a3da-a8f5236901b8', aic=-2300.5850929940457, bic=-2301.198798632926, r2=1.0, rmsd=0.0)), CalibrationModel(id='fc1eedbf-d221-4c0e-a12b-239739727678', name='quadratic', signal_equation='a * s0**2 + b * s0', parameters=[Parameter(id='b38dbe8d-7f86-4043-b3fb-98190b43bdc8', name='a', value=None, init_value=0.1,

CaliPytion contains predefined models, which can be used for calibration.
Alternatively, custom models can be added. 

```{note}
The equation of a model must include 'signal' and 'concentration' as variables.
```

In [ ]:
exponential = calibrator.add_model(
    name="exponential",
    signal_equation="a * exp(b * concentration)",
)

TypeError: Calibrator.add_model() got an unexpected keyword argument 'signal_equation'

By calling `fit_models`, all models are fitted to the data defined in `Standard`. A report, summarizing statistical parameters of each model is generated.

In [ ]:
# Fit all defined models
calibrator.fit_models()

,AIC,R squared,RMSD
Model Name,,,
linear,-197,0.9988,0.0252
quadratic,-196,0.9989,0.0245
cubic,-195,0.9989,0.0242
exponential,-85,0.9303,0.1909


## Model visualization

Models can be visualized, displaying the measured samples used for fitting of thee model, as well as the fitted model. Additionally, the residuals of the model are visualized.

In [ ]:
# Visualize the models
calibrator.visualize()

## Save model to `Standard`

After assessing different models, the best model can be saved to the `Standard` object.

In [ ]:
linear = calibrator.get_model("linear")
print(linear)

CalibrationModel
├── id = calibrationmodel1
├── name = linear
├── equation = a * concentration = signal
├── parameters
│   └── 0
│       └── Parameter
│           ├── id = parameter0
│           ├── name = a
│           ├── value = 0.014053110840428804
│           ├── init_value = 0.1
│           ├── standard_error = 5.1837684365924884e-05
│           ├── lower_bound = -inf
│           └── upper_bound = inf
├── was_fitted = True
├── calibration_range
│   └── CalibrationRange
│       ├── id = calibrationrange4
│       ├── conc_lower = 0.0
│       ├── conc_upper = 160.0
│       ├── signal_lower = -0.023015429423407584
│       └── signal_upper = 2.260977385282519
└── statistics
    └── FitStatistics
        ├── id = fitstatistics4
        ├── aic = -196.83690594773665
        ├── bic = -195.54106908173233
        ├── r2 = 0.9987888099324782
        └── rmsd = 0.02516842805363828



In [ ]:
standard = calibrator.save_model(linear)

## Calculate concentrations

Models can be used to calculate concentrations of unknown signals. 

In [ ]:
linear.calculate([1, 0.1, 2.234])

[71.15862184215769, 7.115862184215769, 158.96836119538025]

If the signal is out of calibration bonds, the calculate method returns `float('nan')` values for the respective signals.

In [ ]:
linear.calculate([1, 100, 5, 0.5])

[71.15862184215769, nan, nan, 35.57931092107884]

In [ ]:
standard.to_animl(
    out_file=f"../data/standard_{str(datetime.now().date())}.animl")